In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pydicom

import pydicom
import os
import glob
import pathlib
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 5.1 MB/s 


In [ ]:
'''
Below is the function to load a scan, which consists of multiple slices in the DICOM format, for the first time. 
'''

def load_scan(path):

    slices = [pydicom.read_file(filename, force=True) for filename in path]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

In [ ]:
'''
Below is the function to resample a scan in order to make slices within it spatially homogeneous. 
Resampling is the basic preprocessing step for all types of bioimages.
'''

def get_img(scans):

    image = np.stack([s.pixel_array for s in scans])
    image = image.astype(np.int16) 
  
    return np.array(image, dtype=np.int16)
    


def resample(image, scan, new_spacing=[1,1,1]): # 1-mm resampling
    
    spacing = np.array([patient[0].SliceThickness]+list(patient[0].PixelSpacing), dtype=np.float32)

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor)
    
    return image, new_spacing

In [ ]:
i = 0
files = []
dir = "/content/drive/MyDrive/..." #insert the path of the scans in dcm format
folders = glob.glob(dir+"/*",recursive=True)
for folder in folders:
  for dirName, subdirList, fileList in os.walk(folder):
    a = glob.glob(dirName+"/*.dcm",recursive=True)
    if a:
      files.append(a)

      path = pathlib.PurePath(str(files))
      last_dir = path.parent.name
      if dirName.endswith(last_dir):
        print("Scan n. "+str(i+1))
        print(dirName)
      i+=1
      
print("----------------------------------------------------")      
print("Total number of PET scans: "+str(len(files))+"")

In [ ]:
i = 0
for file in range (len(files)):
  patient = load_scan(files[i])
  img = get_img(patient)
  print("Scan n. "+str(i+1))
  print("Shape of PET scan before 1-mm resampling: "+str(img.shape)+'')
  resam_scan, spacing = resample(img, patient, [1,1,1])
  print("Shape of PET scan after 1-mm resampling: "+str(resam_scan.shape)+'')
  i+=1
  
  np.save("/content/drive/MyDrive/.../resampled-"+str(i),resam_scan) #insert the path where to save the resampled scans